# Single Particle Model (SPM) 

## Model Equations

The SPM consists of two spherically symmetric diffusion equations: one within a representative negative particle ($k=\text{n}$) and one within a representative positive particle ($k=\text{p}$). In the centre of the particle the classical no-flux condition is imposed. Since the SPM assumes that all particles in an electrode behave in exactly the same way, the flux on the surface of a particle is simply the current divided by the thickness of the electrode. We shall use $r_k\in[0,1]$ for the radial coordinate of the particle in electrode $k$. The concentration of lithium in electrode $k$ is denoted $c_k$ and the current is denoted by $\mathcal{I}$. All parameters in the model stated here are dimensionless and are given in terms of dimensional parameters at the end of this notebook.The model equations for the SPM are then: 
$$
\frac{\partial c_k}{\partial t} = \gamma_k \frac{\partial}{\partial r_k}\left( r_k^2 \frac{\partial c_k}{\partial r_k} \right) \\ 
\frac{\partial c_k}{\partial r_k}\bigg|_{r_k=0} = 0, \quad -\beta_k \hat{C}_k \gamma_k \frac{\partial c_k}{\partial r_k}\bigg|_{r_k=1} = \begin{cases} 
    \frac{\mathcal{I}}{L_n}, \quad k=\text{n}, \\ 
    -\frac{\mathcal{I}}{L_p}, \quad k=\text{p},
\end{cases} ,
$$ 
for $k\in\{\text{n, p}\}$.

### Voltage Expression
The terminal voltage is obtained from the expression: 
$$
V = U_p(c_p)\big|_{r_p=1}-U_n(c_n)\big|_{r_n=1}-\frac{2}{\Lambda}\sinh^{-1}\left(\frac{\mathcal{I}}{g_p L_p} \right)  -\frac{2}{\Lambda}\sinh^{-1}\left(\frac{\mathcal{I}}{g_n L_n} \right)
$$
with the exchange current densities given by
$$
g_k = m_k \hat{C}_k (c_k)^{1/2} (1-c_k)^{1/2}, \quad \text{for } k\in\{\text{n, p}\}.
$$

TODO: define all these symbols

## Example solving SPM using PyBaMM

Below we show how to solve the SPM model, using the default geometry, mesh, paramters, discretisation and solver provided with PyBaMM.

First we need to import pybamm, and then change our working directory to the root of the pybamm folder. 

In [ ]:
import pybamm
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

We then get the SPH model equations:

In [ ]:
model = pybamm.lithium_ion.SPM()

The model object is a subtype of [`pybamm.BaseModel`](https://pybamm.readthedocs.io/en/latest/source/models/base_model.html), and contains all the equations that define this particular model. For example, the `rhs` dict contained in `model` has a dictionary mapping variables such as $c_n$ to the equation representing its rate of change with time (i.e. $\frac{c_n}{dt}$). We can see this explicitly be visualising this entry in the `rhs` dict:

In [ ]:
variable = next(iter(model.rhs.keys()))
equation = next(iter(model.rhs.values()))
print('rhs equation for variable \'',variable,'\' is:')
path = 'examples/notebooks/models/'
equation.visualise(path+'spm1.png')

![](spm1.png)

We need a geometry to define our model equations over. In pybamm this is represented by the [`pybamm.Geometry`](https://pybamm.readthedocs.io/en/latest/source/geometry/geometry.html) class. In this case we use the default geometry object defined by the model

In [ ]:
geometry = model.default_geometry

This geometry object defines a number of geometry domains, each with its own name, spatial variables and min/max limits (the latter are represented as equations similar to the rhs equation shown above). For instance, the SPM model has the following domains:

In [ ]:
print('SPM domains:')
for i, (k, v) in enumerate(geometry.items()):
    print(str(i+1)+'.',k,'with variables:')
    for var, rng in v['primary'].items():
        print('  -(',rng['min'],') <=',var,'<= (',rng['max'],')')

Both the model equations and the geometry are defined by a set of parameters, such as $\gamma_p$ or $L_p$. We can substitute these symbolic parameters in the model with values by using the [`pybamm.ParameterValues`](https://pybamm.readthedocs.io/en/latest/source/parameters/parameter_values.html) class, which takes either a python dictionary or CSV file with the mapping between parameter names and values. Rather than create our own instance of `pybamm.ParameterValues`, we will use the default parameter set included in the model

In [ ]:
param = model.default_parameter_values

We can then apply this parameter set to the model and geometry

In [ ]:
param.process_model(model)
param.process_geometry(geometry)

The next step is to mesh the input geometry. We can do this using the [`pybamm.Mesh`](https://pybamm.readthedocs.io/en/latest/source/meshes/meshes.html) class. This class takes in the geometry of the problem, and also two dictionaries containing the type of mesh to use within each domain of the geometry (i.e. within the positive or negative electrode domains), and the number of mesh points. 

The default mesh types and the default number of points to use in each variable for the SPH model are:

In [ ]:
for k, t in model.default_submesh_types.items():
    print(k,'is of type',t.__name__)
for var, npts in model.default_var_pts.items():
    print(var,'has',npts,'mesh points')

With these defaults, we can then create our mesh of the given geometry:

In [ ]:
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

The next step is to discretise the model equations over this mesh. We do this using the [`pybamm.Discretisation`](https://pybamm.readthedocs.io/en/latest/source/discretisations/discretisation.html) class, which takes both the mesh we have already created, and a dictionary of spatial methods to use for each geometry domain. For the case of the SPM model, we use the following defaults for the spatial discretisation methods:

In [ ]:
for k, method in model.default_spatial_methods.items():
    print(k,'is discretised using',method.__name__,'method')

We then create the `pybamm.Discretisation` object, and use this to discretise the model equations

In [ ]:
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

After this stage, all the equations in `model` have been discretised into purely linear algebra expressions that are ready to be evaluated within a time-stepping loop of a given solver. Once again we use the default ODE solver for this model

In [ ]:
# Solve the model at the given time points
solver = model.default_solver
n = 100
t_eval = np.linspace(0, 1, n)
print('Solving using',type(solver).__name__,'solver...')
solver.solve(model, t_eval)
print('Finished.')

Each model in pybamm has a list of relevent variables defined in the model, for use in visualising the model solution or for comparison with other models. The SPM model defines the following variables:

In [ ]:
print('SPM model variables:')
for v in model.variables.keys():
    print('\t-',v)

To help visualise the results, pybamm provides the `pybamm.ProcessedVariable` class, which takes the output of a solver and a variable, and allows the user to evaluate the value of that variable at any given time or $x$ value. For example, we can create a `pybamm.ProcessedVariable` using the SPM voltage variable, and plot the voltage versus time.

In [ ]:
voltage = pybamm.ProcessedVariable(model.variables['Terminal voltage'], solver.t, solver.y, mesh=mesh)

In [ ]:
t = np.linspace(0,1,100)
plt.plot(t, voltage(t=t).reshape(-1))
plt.xlabel(r'$t$')
plt.ylabel('Terminal voltage')
plt.show()

In [ ]:
c_n = pybamm.ProcessedVariable(model.variables['Negative particle concentration'], solver.t, solver.y, mesh=mesh)
c_p = pybamm.ProcessedVariable(model.variables['Positive particle concentration'], solver.t, solver.y, mesh=mesh)
x = np.linspace(0,1,100)

f, (ax1, ax2) = plt.subplots(1,2)
plot_c_n = ax1.plot(x, c_n(x=x,t=0))
plot_c_p = ax2.plot(x, c_p(x=x,t=0))

def plot_concentrations(t):
    plot_c_n.set_ydata(c_n(x=x, t=t)
    plot_c_p.set_ydata(c_n(x=x, t=t)

    